In [1]:
import optuna
from scipy import stats
import numpy as np

In [21]:
[s.study_name for s in optuna.study.get_all_study_summaries(storage="sqlite://///store-01.hpi.uni-potsdam.de/fg/doellner/florence.boettger/counterfactuals/optimize_counterfactuals_full.db")]

['optimize_counterfactuals_full',
 'optimize_counterfactuals_full_lower_range',
 'optimize_counterfactuals_2',
 'optimize_counterfactuals_vgg_distance',
 'optimize_counterfactuals_vgg_distance_real',
 'optimize_counterfactuals_vgg_distance_final',
 'optimize_counterfactuals_resnet']

In [3]:
study = optuna.load_study(storage="sqlite://///store-01.hpi.uni-potsdam.de/fg/doellner/florence.boettger/counterfactuals/optimize_counterfactuals_full.db", study_name="optimize_counterfactuals_resnet")

In [22]:
# study = optuna.create_study(
#     storage="sqlite://///store-01.hpi.uni-potsdam.de/fg/doellner/florence.boettger/counterfactuals/optimize_counterfactuals_full.db",
#     study_name="optimize_counterfactuals_resnet",
#     directions=["maximize", "minimize"],
# )
# study = optuna.load_study(storage="sqlite://///store-01.hpi.uni-potsdam.de/fg/doellner/florence.boettger/counterfactuals/optimize_counterfactuals_full.db", study_name="optimize_counterfactuals_resnet")
# study.enqueue_trial(
#     {
#         "lambd2": 10,
#         "max_dist": 0
#     }
# )
# study.enqueue_trial(
#     {
#         "lambd2": 0,
#         "max_dist": 0
#     }
# )

In [5]:
len(study.trials)

404

In [6]:
np.average([t.params["max_dist"] for t in study.best_trials])

0.383784542831845

In [7]:
optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"])

In [8]:
figs_dist = [optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"], constraints_func=lambda t: [-1] if t.params["max_dist"] >= n and t.params["max_dist"] < n + 0.5 else [+1]) for n in np.arange(0.0, 3.0, 0.5)]
for fig in figs_dist:
    fig.show()

C:\Users\Flori\AppData\Roaming\Python\Python39\site-packages\optuna\visualization\_pareto_front.py:227: ExperimentalWarning:

``constraints_func`` argument is an experimental feature. The interface can change in the future.



In [9]:
figs_lambd2 = [optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"], constraints_func=lambda t: [-1] if t.params["lambd2"] >= n and t.params["lambd2"] < n + 2.0 else [+1]) for n in np.arange(0.0, 10.0, 2.0)]
for fig in figs_lambd2:
    fig.show()

In [10]:
# fig_additive = optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"], constraints_func=lambda t: [+1] if t.params["mode"] == "additive" else [-1])
# fig_multiplicative = optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"], constraints_func=lambda t: [+1] if t.params["mode"] == "multiplicative" else [-1])

# fig_additive.show()
# fig_multiplicative.show()

In [11]:
# fig_full = optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"], constraints_func=lambda t: [+1] if t.params["parts_type"] == "full" else [-1])
# fig_minimize_head = optuna.visualization.plot_pareto_front(study, target_names=["KP", "edits"], constraints_func=lambda t: [+1] if t.params["parts_type"] == "minimize_head" else [-1])

# fig_full.show()
# fig_minimize_head.show()

In [12]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[1], target_name="edits")

In [13]:
optuna.visualization.plot_optimization_history(study, target=lambda t: t.values[0], target_name="KP")

In [14]:
optuna.visualization.plot_contour(study, params=["lambd2", "max_dist"], target=lambda t: t.values[0], target_name="KP")

In [15]:
optuna.visualization.plot_contour(study, params=["lambd2", "max_dist"], target=lambda t: t.values[1], target_name="edits")

In [16]:
def study_spearman(study, x_names, y_length):
    string_map = {
        "binary": 0,
        "distance": 1,
        "full": 0,
        "minimize_head": 1,
        "additive": 0,
        "multiplicative": 1,
    }
    x = [[string_map[t.params[x_name]] if isinstance(t.params[x_name], str) else t.params[x_name] for x_name in x_names] for t in study.trials if t.values is not None]
    y = [[t.values[y_index] for y_index in range(y_length)] for t in study.trials if t.values is not None]
    return stats.spearmanr(x, y)

In [17]:
x_names = ["lambd2", "max_dist"]
spearman = study_spearman(study, x_names, 2)

In [18]:
for i in range(len(x_names)):
    x_name = x_names[i]
    print(f"{x_name} spearman correlation is: {spearman.correlation[i][len(x_names)]} on KP, {spearman.correlation[i][len(x_names) + 1]} on edits.")
    print(f"{x_name} spearman pvalue is: {spearman.pvalue[i][len(x_names)]} on KP, {spearman.pvalue[i][len(x_names) + 1]} on edits.")

lambd2 spearman correlation is: 0.7682707929031944 on KP, 0.8223198786730311 on edits.
lambd2 spearman pvalue is: 1.0617260069714071e-79 on KP, 3.1071663991628765e-100 on edits.
max_dist spearman correlation is: -0.56943225570079 on KP, -0.4775797174028604 on edits.
max_dist spearman pvalue is: 5.1777682209516554e-36 on KP, 2.3738476571178457e-24 on edits.
